The notebook is meant to reqrite hdf5 files, specifically the COMPAS_Output_wWeights_SM_2.h5 file in order to add WD+WD to the BSE_Double_Compact objects to eventually calculate the rate of WD+WD,

My steps to do this include:,
- Making the DCO table shorter so that it only encompasses things that I actually need,
- Making an empty table that has the same columns as the DCO table,
- Masking the original system parameters so that I only select for WD+WD that merge within a Hubble time,
    - Computing the coalescence time,
    - Checking if the binary merges within a hubble time,
- Adding all the information about these WD+WD to the empty table and then adding this table to the original DCO table,
- Using this new table to create a new COMPAS output to then run fastcosmicintegration on,
- Changing FastCosmisIntegration.py so that BWD are added as a DCO type + ,
- run FastCosmicIntegration via the blackbox method and compare with the Brill Paper Lieke sent"

Let's first look at the columns in BSE_DCOs to see what information we need of the WD+WD to gather

In [1]:
#importing, make sure the kernel is correct or the module will be unknown
import h5py as h5
import pandas as pd
from astropy.table import Table
from astropy import units as u
from astropy import constants as const
import numpy as np
import matplotlib.pyplot as plt

/mnt/sw/nix/store/71ksmx7k6xy3v9ksfkv5mp5kxxp64pd6-python-3.10.13-view/lib/python3.10/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/mnt/sw/nix/store/71ksmx7k6xy3v9ksfkv5mp5kxxp64pd6-python-3.10.13-view/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/mnt/sw/nix/store/71ksmx7k6xy3v9ksfkv5mp5kxxp64pd6-python-3.10.13-view/lib/python3.10/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/mnt/sw/nix/store/71ksmx7k6xy3v9ksfkv5mp5kxxp64pd6-python-3.10.13-view/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <

In [2]:
# reading in the HDF5 file - this file is the AIS file that we should expect many WDs in
pathToWDWD_edit_H5 = '/mnt/home/msantiago/ceph/data/CosmicIntegration/CosmicIntegration/COMPAS_Output_wWeights_NEW_EDIT_1.h5' #path of the hdf5 file
Data_WD_edit = h5.File(pathToWDWD_edit_H5)

In [3]:
# we want to read in the bse_system_paramtetrs to get information about thes different systems
DCO_OG = pd.DataFrame() # making a pandas dataframe
for key in Data_WD_edit["BSE_Double_Compact_Objects"].keys(): #looping through the \"keys\" or paramters in BSE_System_Parameters
    DCO_OG[key] = Data_WD_edit["BSE_Double_Compact_Objects"][key][()] # adding these columns to the dataframe"

In [4]:
pd.set_option('display.max_columns', None) # allows us to see all of the columns
display(DCO_OG)

,CE_Event_Counter,Coalescence_Time,Eccentricity@DCO,Immediate_RLOF>CE,MT_Donor_Hist(1),MT_Donor_Hist(2),Mass(1),Mass(2),Merges_Hubble_Time,Metallicity@ZAMS(1),Optimistic_CE,Record_Type,Recycled_NS(1),Recycled_NS(2),SEED,SemiMajorAxis@DCO,Stellar_Type(1),Stellar_Type(2),Time,dmMT(1),dmMT(2),dmWinds(1),dmWinds(2),mixture_weight
0,0,7.950392e+08,9.042606e-01,0,b'2 ',b'2 ',7.621390,1.525412,0,0.000300,0,1,0,0,395,3.068217,14,13,22.738848,0.000000,0.000000,0.0,-0.000026,9.994028
1,0,1.204001e+18,1.110223e-16,0,b'NA ',b'NA ',15.490306,14.294056,0,0.012003,0,1,0,0,1113,396.078766,14,14,4.874691,0.000000,0.000000,0.0,-0.027284,10.000000
2,0,2.500925e+16,3.330714e-16,0,b'NA ',b'NA ',19.804426,16.017112,0,0.003526,0,1,0,0,1124,172.274778,14,14,5.295735,0.000000,0.000000,0.0,-0.001613,10.000000
3,0,1.419177e+06,1.110223e-16,0,b'2 ',b'2 ',26.873803,34.481029,0,0.002050,0,1,0,0,1229,0.707168,14,14,5.500738,0.000000,0.000000,0.0,0.000000,10.000000
4,1,1.784783e+06,0.000000e+00,0,b'4 ',b'5 ',11.306701,9.799469,0,0.001517,0,1,0,0,1472,0.337254,14,14,9.005598,0.000000,0.000000,0.0,0.000000,10.000000
5,0,5.789181e+18,5.110764e-01,0,b'NA ',b'NA ',15.403925,8.113321,0,0.001195,0,1,0,0,1686,624.503829,14,14,10.242669,0.000000,0.000000,0.0,-0.009240,10.000000
6,0,3.782904e+16,5.940288e-01,0,b'NA ',b'NA ',15.388835,10.652164,0,0.004389,0,1,0,0,2015,218.814812,14,14,7.771299,0.000000,0.000000,0.0,-0.003354,10.000000
7,0,1.552547e+06,1.570092e-16,0,b'2 ',b'2 ',12.448706,16.925402,0,0.027860,0,1,0,0,2316,0.415424,14,14,6.136396,0.000000,0.000000,0.0,-0.063110,10.000000
8,0,6.816599e+17,4.577567e-16,0,b'NA ',b'NA ',28.017859,21.497305,0,0.003593,0,1,0,0,2477,501.020042,14,14,4.367385,0.000000,0.000000,0.0,0.000000,10.000000
9,0,5.698510e+15,4.443598e-01,0,b'NA ',b'NA ',18.929575,7.974971,0,0.000838,0,1,0,0,2606,111.562666,14,14,10.425692,0.000000,0.000000,0.0,-0.009821,10.000000


Let's drop any columns we don't think are necessary to have anymore

In [5]:
DCO_OG = DCO_OG.drop(['dmMT(1)','dmMT(2)','dmWinds(1)','dmWinds(2)'],axis=1)

In [6]:
DCO_OG

,CE_Event_Counter,Coalescence_Time,Eccentricity@DCO,Immediate_RLOF>CE,MT_Donor_Hist(1),MT_Donor_Hist(2),Mass(1),Mass(2),Merges_Hubble_Time,Metallicity@ZAMS(1),Optimistic_CE,Record_Type,Recycled_NS(1),Recycled_NS(2),SEED,SemiMajorAxis@DCO,Stellar_Type(1),Stellar_Type(2),Time,mixture_weight
0,0,7.950392e+08,9.042606e-01,0,b'2 ',b'2 ',7.621390,1.525412,0,0.000300,0,1,0,0,395,3.068217,14,13,22.738848,9.994028
1,0,1.204001e+18,1.110223e-16,0,b'NA ',b'NA ',15.490306,14.294056,0,0.012003,0,1,0,0,1113,396.078766,14,14,4.874691,10.000000
2,0,2.500925e+16,3.330714e-16,0,b'NA ',b'NA ',19.804426,16.017112,0,0.003526,0,1,0,0,1124,172.274778,14,14,5.295735,10.000000
3,0,1.419177e+06,1.110223e-16,0,b'2 ',b'2 ',26.873803,34.481029,0,0.002050,0,1,0,0,1229,0.707168,14,14,5.500738,10.000000
4,1,1.784783e+06,0.000000e+00,0,b'4 ',b'5 ',11.306701,9.799469,0,0.001517,0,1,0,0,1472,0.337254,14,14,9.005598,10.000000
5,0,5.789181e+18,5.110764e-01,0,b'NA ',b'NA ',15.403925,8.113321,0,0.001195,0,1,0,0,1686,624.503829,14,14,10.242669,10.000000
6,0,3.782904e+16,5.940288e-01,0,b'NA ',b'NA ',15.388835,10.652164,0,0.004389,0,1,0,0,2015,218.814812,14,14,7.771299,10.000000
7,0,1.552547e+06,1.570092e-16,0,b'2 ',b'2 ',12.448706,16.925402,0,0.027860,0,1,0,0,2316,0.415424,14,14,6.136396,10.000000
8,0,6.816599e+17,4.577567e-16,0,b'NA ',b'NA ',28.017859,21.497305,0,0.003593,0,1,0,0,2477,501.020042,14,14,4.367385,10.000000
9,0,5.698510e+15,4.443598e-01,0,b'NA ',b'NA ',18.929575,7.974971,0,0.000838,0,1,0,0,2606,111.562666,14,14,10.425692,10.000000


Let's now make an empty dataframe that has the same columns as the dataframe above

In [7]:
dco_column_names = DCO_OG.columns

In [8]:
WD_DCO_TABLE = pd.DataFrame(columns=[dco_column_names[0],dco_column_names[1],dco_column_names[2],dco_column_names[3],dco_column_names[4],dco_column_names[5],dco_column_names[6], \
dco_column_names[7],dco_column_names[8],dco_column_names[9],dco_column_names[10],dco_column_names[11],dco_column_names[12],dco_column_names[13],dco_column_names[14],dco_column_names[15],\
dco_column_names[16],dco_column_names[17],dco_column_names[18],dco_column_names[19]])

Now that we see what columns are needed for BSE_DCOs let's look at BSE_System_Parameters to look at what information we already have

In [9]:
# we want to read in the bse_system_paramtetrs to get information about thes different systems
WD_SP_edit = pd.DataFrame() # making a pandas dataframe

for key in Data_WD_edit["BSE_System_Parameters"].keys(): #looping through the "keys" or paramters in BSE_System_Parameters
    WD_SP_edit[key] = Data_WD_edit["BSE_System_Parameters"][key][()] # adding these columns to the dataframe

In [10]:
pd.set_option('display.max_columns', None) # allows us to see all of the columns
display(WD_SP_edit)

,Applied_Kick_Magnitude(1),Applied_Kick_Magnitude(2),CE_Alpha,CE_Event_Counter,CH_on_MS(1),CH_on_MS(2),Drawn_Kick_Magnitude(1),Drawn_Kick_Magnitude(2),Eccentricity,Eccentricity@ZAMS,Equilibrated_At_Birth,Error,Evolution_Status,Immediate_RLOF>CE,LBV_Factor,MT_Donor_Hist(1),MT_Donor_Hist(2),Mass(1),Mass(2),Mass@ZAMS(1),Mass@ZAMS(2),Merger,Merger_At_Birth,Metallicity@ZAMS(1),Metallicity@ZAMS(2),Omega@ZAMS(1),Omega@ZAMS(2),Optimistic_CE,Record_Type,SEED,SN_Kick_Magnitude_Random_Number(1),SN_Kick_Magnitude_Random_Number(2),SN_Kick_Mean_Anomaly(1),SN_Kick_Mean_Anomaly(2),SN_Kick_Phi(1),SN_Kick_Phi(2),SN_Kick_Theta(1),SN_Kick_Theta(2),SemiMajorAxis,SemiMajorAxis@ZAMS,Sigma_Kick_CCSN_BH,Sigma_Kick_CCSN_NS,Sigma_Kick_ECSN,Sigma_Kick_USSN,Stellar_Type(1),Stellar_Type(2),Stellar_Type@ZAMS(1),Stellar_Type@ZAMS(2),SystemicSpeed,Time,Unbound,WR_Factor,mixture_weight
0,0.0,0.0,1.0,0,0,0,0.0,0.0,0.0,0.0,0,0,15,0,1.5,b'NA ',b'NA ',0.638226,0.614185,1.112211,1.024209,0,0,0.000186,0.000186,0.0,0.0,0,1,0,0.162910,0.282618,5.951440,1.455541,1.155394,0.500548,4.676608,4.774868,187216.969813,510.224496,265.0,265.0,30.0,30.0,11,11,1,1,0.0,5842.090900,0,1.0,5.039797
1,0.0,0.0,1.0,1,0,0,0.0,0.0,0.0,0.0,0,0,12,1,1.5,b'5 ',b'NA ',1.749202,1.630186,5.371650,1.630186,1,0,0.000345,0.000345,0.0,0.0,0,1,1,0.997185,0.720324,5.859431,0.000719,-0.406432,-0.556034,6.277157,0.580181,101.379276,3.040819,265.0,265.0,30.0,30.0,9,1,1,1,0.0,86.103080,0,1.0,0.674882
2,0.0,0.0,1.0,1,0,0,0.0,0.0,0.0,0.0,0,0,3,0,1.5,b'3 ',b'NA ',0.417609,0.751798,1.552801,0.751798,0,0,0.000620,0.000620,0.0,0.0,0,1,2,0.185082,0.025926,5.853044,3.453631,-0.129719,-0.367118,3.045768,2.075555,6.608342,0.498800,265.0,265.0,30.0,30.0,10,1,1,1,0.0,13717.376578,0,1.0,9.758861
3,0.0,0.0,1.0,0,0,0,0.0,0.0,0.0,0.0,0,0,15,0,1.5,b'NA ',b'NA ',1.065271,0.670867,2.624303,1.209345,0,0,0.000115,0.000115,0.0,0.0,0,1,3,0.070725,0.708148,5.277555,1.827808,0.021657,-0.126211,3.577089,5.631576,56976.149911,119.709655,265.0,265.0,30.0,30.0,11,11,1,1,0.0,3290.631337,0,1.0,0.172821
4,0.0,0.0,1.0,0,0,0,0.0,0.0,0.0,0.0,0,0,15,0,1.5,b'NA ',b'NA ',1.066278,0.641769,2.628817,1.123310,0,0,0.000194,0.000194,0.0,0.0,0,1,4,0.900621,0.547232,1.085077,6.111556,0.444085,0.196372,3.826684,1.357730,9799.324534,20.694780,265.0,265.0,30.0,30.0,11,11,1,1,0.0,4318.463259,0,1.0,0.216670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0.0,0.0,1.0,0,0,0,0.0,0.0,0.0,0.0,0,0,15,0,1.5,b'6 ',b'NA ',0.877053,0.788728,2.772065,1.835501,0,0,0.004224,0.004224,0.0,0.0,0,1,99995,0.636286,0.891204,1.305430,0.835666,-0.234848,0.344817,0.614040,3.624584,3410.571818,3.882745,265.0,265.0,30.0,30.0,11,11,1,1,0.0,1411.164121,0,1.0,0.273067
99996,0.0,0.0,1.0,0,0,0,0.0,0.0,0.0,0.0,0,0,15,0,1.5,b'NA ',b'NA ',1.106444,1.097736,3.367002,2.853142,0,0,0.000123,0.000123,0.0,0.0,0,1,99996,0.470017,0.174221,0.457951,0.348183,-0.177768,0.053696,5.865884,4.063802,13840.821373,22.712985,265.0,265.0,30.0,30.0,11,11,1,1,0.0,354.066004,0,1.0,0.118433
99997,0.0,0.0,1.0,0,0,0,0.0,0.0,0.0,0.0,0,0,15,0,1.5,b'NA ',b'NA ',0.824877,0.781691,2.712533,2.509096,0,0,0.002747,0.002747,0.0,0.0,0,1,99997,0.207631,0.496630,0.753043,1.061202,1.187717,0.988819,0.589580,2.080383,44211.404740,62.900006,265.0,265.0,30.0,30.0,11,11,1,1,0.0,587.286296,0,1.0,0.137214
99998,0.0,0.0,1.0,0,0,0,0.0,0.0,0.0,0.0,0,0,15,0,1.5,b'NA ',b'NA ',0.814940,0.551306,2.014314,0.910640,0,0,0.004745,0.004745,0.0,0.0,0,1,99998,0.150930,0.462669,4.229695,0.470100,-0.369875,-0.662262,1.183951,2.827022,2548.104134,5.524309,265.0,265.0,30.0,30.0,11,11,1,1,0.0,11942.559794,0,1.0,1.299077


The columns that I do not need to calculate the coalescence time for the WD+WD or do not need for the DCO table:

- Applied_Kick_Magnitude(1) & Applied_Kick_Magnitude(2)
- CE_Alpha
- CH_on_MS(1) & CH_on_MS(2)
- Drawn_Kick_Magnitude(1) & Drawn_Kick_Magnitude(2)
- Equilibrated_At_Birth
- Error
- Evolution_Status
- LBV_Factor
- Mass@ZAMS(1) & Mass@ZAMS(2)
- Merger
- Merger_At_Birth
- Metallicity@ZAMS(2)
- Omega@ZAMS(1) & Omega@ZAMS(2)
- SN_Kick_Magnitude_Random_Number(1) & SN_Kick_Magnitude_Random_Number(2)
- SN_Kick_Mean_Anomaly(1) & SN_Kick_Mean_Anomaly(2)
- SN_Kick_Phi(1) & SN_Kick_Phi(2)
- SN_Kick_Theta(1) & SN_Kick_Theta(2)
- SemiMajorAxis@ZAMS
- Sigma_Kick_CCSN_BH
- Sigma_Kick_CCSN_NS
- Sigma_Kick_ECSN
- Sigma_Kick_USSN
- Stellar_Type@ZAMS(1) & Stellar_Type@ZAMS(2)
- SystemicSpeed
- Unbound

In the DCO table the @DCO marking is the same as the final state of the WD+WD"

In [11]:
WD_SP_edit = WD_SP_edit.drop(['Applied_Kick_Magnitude(1)','Applied_Kick_Magnitude(2)','CE_Alpha','CH_on_MS(1)', \
'CH_on_MS(2)','Drawn_Kick_Magnitude(1)','Drawn_Kick_Magnitude(2)','Equilibrated_At_Birth','Error','Evolution_Status',\
'Eccentricity@ZAMS','LBV_Factor','Mass@ZAMS(1)','Mass@ZAMS(2)','Merger_At_Birth','Metallicity@ZAMS(2)','Omega@ZAMS(1)','Omega@ZAMS(2)',\
'SN_Kick_Magnitude_Random_Number(1)','SN_Kick_Magnitude_Random_Number(2)','SN_Kick_Mean_Anomaly(1)','SN_Kick_Mean_Anomaly(2)', \
'SN_Kick_Phi(1)','SN_Kick_Phi(2)','SN_Kick_Theta(1)','SN_Kick_Theta(2)','SemiMajorAxis@ZAMS','Sigma_Kick_CCSN_BH','Sigma_Kick_CCSN_NS','Sigma_Kick_ECSN','Sigma_Kick_USSN','Stellar_Type@ZAMS(1)','Stellar_Type@ZAMS(2)','SystemicSpeed','Unbound','WR_Factor'],axis=1)

In [12]:
WD_SP_edit

,CE_Event_Counter,Eccentricity,Immediate_RLOF>CE,MT_Donor_Hist(1),MT_Donor_Hist(2),Mass(1),Mass(2),Merger,Metallicity@ZAMS(1),Optimistic_CE,Record_Type,SEED,SemiMajorAxis,Stellar_Type(1),Stellar_Type(2),Time,mixture_weight
0,0,0.0,0,b'NA ',b'NA ',0.638226,0.614185,0,0.000186,0,1,0,187216.969813,11,11,5842.090900,5.039797
1,1,0.0,1,b'5 ',b'NA ',1.749202,1.630186,1,0.000345,0,1,1,101.379276,9,1,86.103080,0.674882
2,1,0.0,0,b'3 ',b'NA ',0.417609,0.751798,0,0.000620,0,1,2,6.608342,10,1,13717.376578,9.758861
3,0,0.0,0,b'NA ',b'NA ',1.065271,0.670867,0,0.000115,0,1,3,56976.149911,11,11,3290.631337,0.172821
4,0,0.0,0,b'NA ',b'NA ',1.066278,0.641769,0,0.000194,0,1,4,9799.324534,11,11,4318.463259,0.216670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0,0.0,0,b'6 ',b'NA ',0.877053,0.788728,0,0.004224,0,1,99995,3410.571818,11,11,1411.164121,0.273067
99996,0,0.0,0,b'NA ',b'NA ',1.106444,1.097736,0,0.000123,0,1,99996,13840.821373,11,11,354.066004,0.118433
99997,0,0.0,0,b'NA ',b'NA ',0.824877,0.781691,0,0.002747,0,1,99997,44211.404740,11,11,587.286296,0.137214
99998,0,0.0,0,b'NA ',b'NA ',0.814940,0.551306,0,0.004745,0,1,99998,2548.104134,11,11,11942.559794,1.299077


Let's select for systems that do not experience a stellar merger just to make sure that we are draling with systems that experience a merger due to gravitational waves

In [15]:
stellar_merger_false = WD_SP_edit['Merger']==False
WD_SP_edit_GW = WD_SP_edit[stellar_merger_false]

This is four less columns than the DCO table because I must add the coalescence time column as well as the merges in hubble time column (and we do not have recycled NS information)

#### Let's calculate the coalescence time and merges hubble time columns

Before doing this though we need to make sure we are only calculating these parameters for WD+WD

In [16]:
def WD_BINARY_BOOLS(dataframe):

# let's first look at if there are only helium white dwarf WD binaries
    HeWD_bool = np.logical_and(dataframe['Stellar_Type(1)']==10,dataframe['Stellar_Type(2)']==10)
# then carbon oxygen WD
    COWD_bool = np.logical_and(dataframe['Stellar_Type(1)']==11,dataframe['Stellar_Type(2)']==11)
# then oxgen neon WD
    ONeWD_bool = np.logical_and(dataframe['Stellar_Type(1)']==12,dataframe['Stellar_Type(2)']==12)

# let's look at the combination of WD binaries

# Helium WD combos
    HeCOWD_bool = np.logical_and(dataframe['Stellar_Type(1)']==10,dataframe['Stellar_Type(2)']==11)
    HeONeWD_bool = np.logical_and(dataframe['Stellar_Type(1)']==10,dataframe['Stellar_Type(2)']==12)

# Carbon Oxygen WD combos
    COHeWD_bool = np.logical_and(dataframe['Stellar_Type(1)']==11,dataframe['Stellar_Type(2)']==10)
    COONeWD_bool = np.logical_and(dataframe['Stellar_Type(1)']==11,dataframe['Stellar_Type(2)']==12)

# Oxygen Neon WD combos
    ONeHeWD_bool = np.logical_and(dataframe['Stellar_Type(1)']==12,dataframe['Stellar_Type(2)']==10)
    ONeCOWD_bool = np.logical_and(dataframe['Stellar_Type(1)']==12,dataframe['Stellar_Type(2)']==11)

    # let's return all of these bools
    return(HeWD_bool,COWD_bool,ONeWD_bool,HeCOWD_bool,HeONeWD_bool,COHeWD_bool,COONeWD_bool,ONeHeWD_bool,ONeCOWD_bool)


In [17]:
# let's now create a dataframe where the above criteria is met based on all of the bools

WDWD_EDIT_SYS = WD_SP_edit_GW.loc[(WD_BINARY_BOOLS(WD_SP_edit_GW)[0])|(WD_BINARY_BOOLS(WD_SP_edit_GW)[1])|(WD_BINARY_BOOLS(WD_SP_edit_GW)[2])|(WD_BINARY_BOOLS(WD_SP_edit_GW)[3])|(WD_BINARY_BOOLS(WD_SP_edit_GW)[4])|(WD_BINARY_BOOLS(WD_SP_edit_GW)[5])|(WD_BINARY_BOOLS(WD_SP_edit_GW)[6])|(WD_BINARY_BOOLS(WD_SP_edit_GW)[7])|(WD_BINARY_BOOLS(WD_SP_edit_GW)[8])]
WDWD_EDIT_SYS

,CE_Event_Counter,Eccentricity,Immediate_RLOF>CE,MT_Donor_Hist(1),MT_Donor_Hist(2),Mass(1),Mass(2),Merger,Metallicity@ZAMS(1),Optimistic_CE,Record_Type,SEED,SemiMajorAxis,Stellar_Type(1),Stellar_Type(2),Time,mixture_weight
0,0,0.0,0,b'NA ',b'NA ',0.638226,0.614185,0,0.000186,0,1,0,187216.969813,11,11,5842.090900,5.039797
3,0,0.0,0,b'NA ',b'NA ',1.065271,0.670867,0,0.000115,0,1,3,56976.149911,11,11,3290.631337,0.172821
4,0,0.0,0,b'NA ',b'NA ',1.066278,0.641769,0,0.000194,0,1,4,9799.324534,11,11,4318.463259,0.216670
6,0,0.0,0,b'6 ',b'NA ',0.726286,0.733475,0,0.001819,0,1,6,8065.848502,11,11,764.414536,0.228002
8,0,0.0,0,b'NA ',b'NA ',0.800165,0.629663,0,0.002102,0,1,8,24212.150979,11,11,1205.345463,0.206714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0,0.0,0,b'6 ',b'NA ',0.877053,0.788728,0,0.004224,0,1,99995,3410.571818,11,11,1411.164121,0.273067
99996,0,0.0,0,b'NA ',b'NA ',1.106444,1.097736,0,0.000123,0,1,99996,13840.821373,11,11,354.066004,0.118433
99997,0,0.0,0,b'NA ',b'NA ',0.824877,0.781691,0,0.002747,0,1,99997,44211.404740,11,11,587.286296,0.137214
99998,0,0.0,0,b'NA ',b'NA ',0.814940,0.551306,0,0.004745,0,1,99998,2548.104134,11,11,11942.559794,1.299077


In the equation for coalescence time, M1 refers to the more massive star whereas M2 refers to the less massive one. I need to make new columns (witht eh help of Lieke) to find these more massive and less massive stars in each bianry. Let's do a check to see if our colaescence times match the COMPAS calculations

In [18]:
def tgw(a,e,Mmoremass,Mlessmass,Data,key,parameter):

    """
    Calcualte the coalescence time (inspiral time)
    a = semi major axis (expected in AU or Rsun)
    e = eccentricity
    Mmoremass = mass of the more massive compact object (expected in solar masses!)
    Mlessmass = mass of the less massive compact object (expected in solar masses!)
    """

    SYS = Data[key]
    sep_unit = SYS[parameter].attrs['units']
    if sep_unit == b'Rsol':
        a = (a * u.Rsun).to(u.m)

    elif sep_unit == b'AU':
        a = (a * u.AU).to(u.m)      

    Mmoremass = (Mmoremass * u.Msun).to(u.kg)
    Mlessmass = (Mlessmass * u.Msun).to(u.kg)

    tc = ((((5*((a)**4)*(const.c**5))/(256*(const.G**3)*(Mmoremass)*(Mlessmass)*((Mmoremass)+(Mlessmass))))*(1+(0.27*e**10)+(0.33*e**20)+(0.2*e**1000))*(1-(e**2))**(7/2)))*((3.171e-8)*(u.yr/u.s))*((1e-6)*(u.Myr/u.yr))

    return tc.values

In [19]:
# unit_dict = {b'Rsol':u.Rsun,
#              b'AU':u.AU}

# print(unit_dict[b'AU'])

In [20]:
# let's first look at the units of some parameters to see if anything must be converted
SPs_WD = Data_WD_edit['BSE_System_Parameters']
print(SPs_WD['Mass(1)'].attrs['units']) 
print(SPs_WD['SemiMajorAxis'].attrs['units'])
print(SPs_WD['Time'].attrs['units'])

b'Msol'
b'Rsol'
b'Myr'


In [21]:
# let's first look at the units of some parameters to see if anything must be converted
dcos_WD = Data_WD_edit['BSE_Double_Compact_Objects']
print(dcos_WD['Mass(1)'].attrs['units']) 
print(dcos_WD['SemiMajorAxis@DCO'].attrs['units'])
print(dcos_WD['Time'].attrs['units'])

b'Msol'
b'AU'
b'Myr'


In [22]:
# # looking at how accurate the coaslescence calulations are
# mergingDCO = DCO_OG[DCO_OG['Merges_Hubble_Time'] == 1]

# # Add columns for the more and less massive compact object
# mergingDCO['M_moremass'] = mergingDCO[['Mass(1)', 'Mass(2)']].max(axis=1)
# mergingDCO['M_lessmass'] = mergingDCO[['Mass(1)', 'Mass(2)']].min(axis=1)

# time_gw = tgw((mergingDCO['SemiMajorAxis@DCO'].values),(mergingDCO['Eccentricity@DCO']),(mergingDCO['M_moremass'].values),(mergingDCO['M_lessmass'].values),Data_WD_edit,'BSE_Double_Compact_Objects','SemiMajorAxis@DCO')

# plt.scatter(mergingDCO['SEED'], abs(time_gw - mergingDCO['Coalescence_Time']) )
# plt.xlabel("SEED")
# plt.ylabel('absolute difference calc t_col and COMPAS Coalescence_Time ')
# plt.yscale('log')
# plt.title("Difference in Computation having separations = [Rsun]")
# plt.show()

#### Let's now calculated the coalescence time to our table of WD+WD

In [23]:
# Add columns for the more and less massive compact object

WDWD_EDIT_SYS['M_moremass'] = WDWD_EDIT_SYS[['Mass(1)', 'Mass(2)']].max(axis=1)
WDWD_EDIT_SYS['M_lessmass'] = WDWD_EDIT_SYS[['Mass(1)', 'Mass(2)']].min(axis=1)

/tmp/ipykernel_1681931/3822573989.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  WDWD_EDIT_SYS['M_moremass'] = WDWD_EDIT_SYS[['Mass(1)', 'Mass(2)']].max(axis=1)
/tmp/ipykernel_1681931/3822573989.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  WDWD_EDIT_SYS['M_lessmass'] = WDWD_EDIT_SYS[['Mass(1)', 'Mass(2)']].min(axis=1)


In [24]:
time_col_wd = tgw((WDWD_EDIT_SYS['SemiMajorAxis'].values),(WDWD_EDIT_SYS['Eccentricity']),(WDWD_EDIT_SYS['M_moremass'].values),(WDWD_EDIT_SYS['M_lessmass'].values),Data_WD_edit,'BSE_System_Parameters','SemiMajorAxis')

In [25]:
# let's insert the check if there was a merger before hubble time into our dataframe

# only run this cell once per table creation
WDWD_EDIT_SYS.insert(1,"Coalescence_Time",time_col_wd,True)
WDWD_EDIT_SYS

,CE_Event_Counter,Coalescence_Time,Eccentricity,Immediate_RLOF>CE,MT_Donor_Hist(1),MT_Donor_Hist(2),Mass(1),Mass(2),Merger,Metallicity@ZAMS(1),Optimistic_CE,Record_Type,SEED,SemiMajorAxis,Stellar_Type(1),Stellar_Type(2),Time,mixture_weight,M_moremass,M_lessmass
0,0,3.761352e+23,0.0,0,b'NA ',b'NA ',0.638226,0.614185,0,0.000186,0,1,0,187216.969813,11,11,5842.090900,5.039797,0.638226,0.614185
3,0,1.276658e+21,0.0,0,b'NA ',b'NA ',1.065271,0.670867,0,0.000115,0,1,3,56976.149911,11,11,3290.631337,0.172821,1.065271,0.670867
4,0,1.185820e+18,0.0,0,b'NA ',b'NA ',1.066278,0.641769,0,0.000194,0,1,4,9799.324534,11,11,4318.463259,0.216670,1.066278,0.641769
6,0,8.181037e+17,0.0,0,b'6 ',b'NA ',0.726286,0.733475,0,0.001819,0,1,6,8065.848502,11,11,764.414536,0.228002,0.733475,0.726286
8,0,7.170428e+19,0.0,0,b'NA ',b'NA ',0.800165,0.629663,0,0.002102,0,1,8,24212.150979,11,11,1205.345463,0.206714,0.800165,0.629663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0,1.764900e+16,0.0,0,b'6 ',b'NA ',0.877053,0.788728,0,0.004224,0,1,99995,3410.571818,11,11,1411.164121,0.273067,0.877053,0.788728
99996,0,2.060422e+18,0.0,0,b'NA ',b'NA ',1.106444,1.097736,0,0.000123,0,1,99996,13840.821373,11,11,354.066004,0.118433,1.106444,1.097736
99997,0,5.543663e+20,0.0,0,b'NA ',b'NA ',0.824877,0.781691,0,0.002747,0,1,99997,44211.404740,11,11,587.286296,0.137214,0.824877,0.781691
99998,0,1.032296e+16,0.0,0,b'NA ',b'NA ',0.814940,0.551306,0,0.004745,0,1,99998,2548.104134,11,11,11942.559794,1.299077,0.814940,0.551306


In [26]:
# in case you run the cell above more than once here is a way to remove the extra column
# WDWD_EDIT_SYS = WDWD_EDIT_SYS.drop('Coalescence_Time', axis=1)

We now need to calcualte the delay time to then see if this time is within a hubble time

In [27]:
# let's convert the columns of the data frame into numpy arrays to be able to do operations with them
time = np.array(WDWD_EDIT_SYS['Time'])
t_col = np.array(WDWD_EDIT_SYS['Coalescence_Time'])

t_delay = [] # this will hold the delay time in Myr

for binaries in range(WDWD_EDIT_SYS.shape[0]):

    delay = time[binaries] + t_col[binaries] # adding the two values together to get the delay time
    t_delay.append(delay)

In [28]:
# first let's take the delay time column and make it a numpy array
delay_time = np.array(t_delay)

# the age of the universe
age_universe = (13.7e9)*(1e-6) # converting from yr to Myr

hubble_merger = []

for times in delay_time:

    if (times > age_universe):
        hubble_merger.append(0)

    elif (times < age_universe):
        hubble_merger.append(1)

In [29]:
# let's insert the check if there was a merger before hubble time into our dataframe

# only run this cell once per table creation
WDWD_EDIT_SYS.insert(8,"Merges_Hubble_Time",hubble_merger,True)

In [30]:
# in case you need to remove the column
# WDWD_SYS = WDWD_SYS.drop('Merges_Hubble_Time', axis=1)

In [31]:
WDWD_EDIT_SYS

,CE_Event_Counter,Coalescence_Time,Eccentricity,Immediate_RLOF>CE,MT_Donor_Hist(1),MT_Donor_Hist(2),Mass(1),Mass(2),Merges_Hubble_Time,Merger,Metallicity@ZAMS(1),Optimistic_CE,Record_Type,SEED,SemiMajorAxis,Stellar_Type(1),Stellar_Type(2),Time,mixture_weight,M_moremass,M_lessmass
0,0,3.761352e+23,0.0,0,b'NA ',b'NA ',0.638226,0.614185,0,0,0.000186,0,1,0,187216.969813,11,11,5842.090900,5.039797,0.638226,0.614185
3,0,1.276658e+21,0.0,0,b'NA ',b'NA ',1.065271,0.670867,0,0,0.000115,0,1,3,56976.149911,11,11,3290.631337,0.172821,1.065271,0.670867
4,0,1.185820e+18,0.0,0,b'NA ',b'NA ',1.066278,0.641769,0,0,0.000194,0,1,4,9799.324534,11,11,4318.463259,0.216670,1.066278,0.641769
6,0,8.181037e+17,0.0,0,b'6 ',b'NA ',0.726286,0.733475,0,0,0.001819,0,1,6,8065.848502,11,11,764.414536,0.228002,0.733475,0.726286
8,0,7.170428e+19,0.0,0,b'NA ',b'NA ',0.800165,0.629663,0,0,0.002102,0,1,8,24212.150979,11,11,1205.345463,0.206714,0.800165,0.629663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0,1.764900e+16,0.0,0,b'6 ',b'NA ',0.877053,0.788728,0,0,0.004224,0,1,99995,3410.571818,11,11,1411.164121,0.273067,0.877053,0.788728
99996,0,2.060422e+18,0.0,0,b'NA ',b'NA ',1.106444,1.097736,0,0,0.000123,0,1,99996,13840.821373,11,11,354.066004,0.118433,1.106444,1.097736
99997,0,5.543663e+20,0.0,0,b'NA ',b'NA ',0.824877,0.781691,0,0,0.002747,0,1,99997,44211.404740,11,11,587.286296,0.137214,0.824877,0.781691
99998,0,1.032296e+16,0.0,0,b'NA ',b'NA ',0.814940,0.551306,0,0,0.004745,0,1,99998,2548.104134,11,11,11942.559794,1.299077,0.814940,0.551306


Let's look at where the \"Merges_Hubble_Time\" is set to true

In [32]:
# there is only one system that merges before hubble time
WDWD_Merge = WDWD_EDIT_SYS[WDWD_EDIT_SYS['Merges_Hubble_Time']==True]
WDWD_Merge

,CE_Event_Counter,Coalescence_Time,Eccentricity,Immediate_RLOF>CE,MT_Donor_Hist(1),MT_Donor_Hist(2),Mass(1),Mass(2),Merges_Hubble_Time,Merger,Metallicity@ZAMS(1),Optimistic_CE,Record_Type,SEED,SemiMajorAxis,Stellar_Type(1),Stellar_Type(2),Time,mixture_weight,M_moremass,M_lessmass
14,1,7781.577103,0.0,0,b'2 ',b'5-8 ',0.571620,0.673707,1,0,0.021682,0,1,14,2.232218,11,11,405.370186,1.578707,0.673707,0.571620
31,1,1373.314685,0.0,0,b'2-8 ',b'5-8 ',0.773915,0.674927,1,0,0.004714,0,1,31,1.621585,11,11,494.155681,0.309698,0.773915,0.674927
40,1,64.020795,0.0,0,b'1-2 ',b'3 ',0.128751,0.345038,1,0,0.001365,0,1,40,0.307706,10,10,4992.556501,9.999993,0.345038,0.128751
52,1,233.593141,0.0,0,b'3 ',b'6 ',0.312582,0.549920,1,0,0.000292,0,1,52,0.692827,10,11,3276.834166,9.371884,0.549920,0.312582
114,1,285.668079,0.0,0,b'1-2 ',b'3 ',0.155492,0.353392,1,0,0.000230,0,1,114,0.480138,10,10,7039.419315,9.999954,0.353392,0.155492
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99883,2,1533.766025,0.0,0,b'6 ',b'3 ',1.328178,0.293608,1,0,0.001031,0,1,99883,1.593854,12,10,1591.331097,0.623121,1.328178,0.293608
99900,2,349.856371,0.0,0,b'5 ',b'3 ',0.587863,0.313331,1,0,0.023187,0,1,99900,0.788408,11,11,2223.960578,0.203871,0.587863,0.313331
99917,2,1005.769001,0.0,0,b'5 ',b'3 ',0.672699,0.305973,1,0,0.011928,0,1,99917,1.077492,11,10,1245.184712,0.559022,0.672699,0.305973
99943,1,12040.174873,0.0,0,b'2 ',b'5 ',0.593881,0.608892,1,0,0.001694,0,1,99943,2.429508,11,11,1121.480719,1.292016,0.608892,0.593881


In [34]:
# making sure all of these systems do not experience a stellar merger
bool_merge = WDWD_Merge['Merger']==False
sum(bool_merge)

2053

In [35]:
# let's drop the merger column
WDWD_Merge = WDWD_Merge.drop('Merger', axis=1)

#### Let's add our WD+WD that merged within a hubble time to the hdf5 file

Before concatenating the WD+WD to the original DCO table we need to rename some of the columns in our WD table so that they match up with the DCO table

In [36]:
WDWD_Merge = WDWD_Merge.rename(columns={'Eccentricity':'Eccentricity@DCO','SemiMajorAxis':'SemiMajorAxis@DCO'})

In [37]:
WDWD_Merge

,CE_Event_Counter,Coalescence_Time,Eccentricity@DCO,Immediate_RLOF>CE,MT_Donor_Hist(1),MT_Donor_Hist(2),Mass(1),Mass(2),Merges_Hubble_Time,Metallicity@ZAMS(1),Optimistic_CE,Record_Type,SEED,SemiMajorAxis@DCO,Stellar_Type(1),Stellar_Type(2),Time,mixture_weight,M_moremass,M_lessmass
14,1,7781.577103,0.0,0,b'2 ',b'5-8 ',0.571620,0.673707,1,0.021682,0,1,14,2.232218,11,11,405.370186,1.578707,0.673707,0.571620
31,1,1373.314685,0.0,0,b'2-8 ',b'5-8 ',0.773915,0.674927,1,0.004714,0,1,31,1.621585,11,11,494.155681,0.309698,0.773915,0.674927
40,1,64.020795,0.0,0,b'1-2 ',b'3 ',0.128751,0.345038,1,0.001365,0,1,40,0.307706,10,10,4992.556501,9.999993,0.345038,0.128751
52,1,233.593141,0.0,0,b'3 ',b'6 ',0.312582,0.549920,1,0.000292,0,1,52,0.692827,10,11,3276.834166,9.371884,0.549920,0.312582
114,1,285.668079,0.0,0,b'1-2 ',b'3 ',0.155492,0.353392,1,0.000230,0,1,114,0.480138,10,10,7039.419315,9.999954,0.353392,0.155492
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99883,2,1533.766025,0.0,0,b'6 ',b'3 ',1.328178,0.293608,1,0.001031,0,1,99883,1.593854,12,10,1591.331097,0.623121,1.328178,0.293608
99900,2,349.856371,0.0,0,b'5 ',b'3 ',0.587863,0.313331,1,0.023187,0,1,99900,0.788408,11,11,2223.960578,0.203871,0.587863,0.313331
99917,2,1005.769001,0.0,0,b'5 ',b'3 ',0.672699,0.305973,1,0.011928,0,1,99917,1.077492,11,10,1245.184712,0.559022,0.672699,0.305973
99943,1,12040.174873,0.0,0,b'2 ',b'5 ',0.593881,0.608892,1,0.001694,0,1,99943,2.429508,11,11,1121.480719,1.292016,0.608892,0.593881


In [38]:
DCO_appended_table = pd.concat([DCO_OG,WDWD_Merge])

In [39]:
DCO_appended_table

,CE_Event_Counter,Coalescence_Time,Eccentricity@DCO,Immediate_RLOF>CE,MT_Donor_Hist(1),MT_Donor_Hist(2),Mass(1),Mass(2),Merges_Hubble_Time,Metallicity@ZAMS(1),Optimistic_CE,Record_Type,Recycled_NS(1),Recycled_NS(2),SEED,SemiMajorAxis@DCO,Stellar_Type(1),Stellar_Type(2),Time,mixture_weight,M_moremass,M_lessmass
0,0,7.950392e+08,9.042606e-01,0,b'2 ',b'2 ',7.621390,1.525412,0,0.000300,0,1,0.0,0.0,395,3.068217,14,13,22.738848,9.994028,NaN,NaN
1,0,1.204001e+18,1.110223e-16,0,b'NA ',b'NA ',15.490306,14.294056,0,0.012003,0,1,0.0,0.0,1113,396.078766,14,14,4.874691,10.000000,NaN,NaN
2,0,2.500925e+16,3.330714e-16,0,b'NA ',b'NA ',19.804426,16.017112,0,0.003526,0,1,0.0,0.0,1124,172.274778,14,14,5.295735,10.000000,NaN,NaN
3,0,1.419177e+06,1.110223e-16,0,b'2 ',b'2 ',26.873803,34.481029,0,0.002050,0,1,0.0,0.0,1229,0.707168,14,14,5.500738,10.000000,NaN,NaN
4,1,1.784783e+06,0.000000e+00,0,b'4 ',b'5 ',11.306701,9.799469,0,0.001517,0,1,0.0,0.0,1472,0.337254,14,14,9.005598,10.000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99883,2,1.533766e+03,0.000000e+00,0,b'6 ',b'3 ',1.328178,0.293608,1,0.001031,0,1,NaN,NaN,99883,1.593854,12,10,1591.331097,0.623121,1.328178,0.293608
99900,2,3.498564e+02,0.000000e+00,0,b'5 ',b'3 ',0.587863,0.313331,1,0.023187,0,1,NaN,NaN,99900,0.788408,11,11,2223.960578,0.203871,0.587863,0.313331
99917,2,1.005769e+03,0.000000e+00,0,b'5 ',b'3 ',0.672699,0.305973,1,0.011928,0,1,NaN,NaN,99917,1.077492,11,10,1245.184712,0.559022,0.672699,0.305973
99943,1,1.204017e+04,0.000000e+00,0,b'2 ',b'5 ',0.593881,0.608892,1,0.001694,0,1,NaN,NaN,99943,2.429508,11,11,1121.480719,1.292016,0.608892,0.593881


In [40]:
# let's drop the extra 
DCO_appended_table = DCO_appended_table.drop('M_moremass', axis=1)
DCO_appended_table = DCO_appended_table.drop('M_lessmass', axis=1)

In [41]:
DCO_appended_table

,CE_Event_Counter,Coalescence_Time,Eccentricity@DCO,Immediate_RLOF>CE,MT_Donor_Hist(1),MT_Donor_Hist(2),Mass(1),Mass(2),Merges_Hubble_Time,Metallicity@ZAMS(1),Optimistic_CE,Record_Type,Recycled_NS(1),Recycled_NS(2),SEED,SemiMajorAxis@DCO,Stellar_Type(1),Stellar_Type(2),Time,mixture_weight
0,0,7.950392e+08,9.042606e-01,0,b'2 ',b'2 ',7.621390,1.525412,0,0.000300,0,1,0.0,0.0,395,3.068217,14,13,22.738848,9.994028
1,0,1.204001e+18,1.110223e-16,0,b'NA ',b'NA ',15.490306,14.294056,0,0.012003,0,1,0.0,0.0,1113,396.078766,14,14,4.874691,10.000000
2,0,2.500925e+16,3.330714e-16,0,b'NA ',b'NA ',19.804426,16.017112,0,0.003526,0,1,0.0,0.0,1124,172.274778,14,14,5.295735,10.000000
3,0,1.419177e+06,1.110223e-16,0,b'2 ',b'2 ',26.873803,34.481029,0,0.002050,0,1,0.0,0.0,1229,0.707168,14,14,5.500738,10.000000
4,1,1.784783e+06,0.000000e+00,0,b'4 ',b'5 ',11.306701,9.799469,0,0.001517,0,1,0.0,0.0,1472,0.337254,14,14,9.005598,10.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99883,2,1.533766e+03,0.000000e+00,0,b'6 ',b'3 ',1.328178,0.293608,1,0.001031,0,1,NaN,NaN,99883,1.593854,12,10,1591.331097,0.623121
99900,2,3.498564e+02,0.000000e+00,0,b'5 ',b'3 ',0.587863,0.313331,1,0.023187,0,1,NaN,NaN,99900,0.788408,11,11,2223.960578,0.203871
99917,2,1.005769e+03,0.000000e+00,0,b'5 ',b'3 ',0.672699,0.305973,1,0.011928,0,1,NaN,NaN,99917,1.077492,11,10,1245.184712,0.559022
99943,1,1.204017e+04,0.000000e+00,0,b'2 ',b'5 ',0.593881,0.608892,1,0.001694,0,1,NaN,NaN,99943,2.429508,11,11,1121.480719,1.292016


In [42]:
DCO_appended_table.sort_values(by="SEED")

,CE_Event_Counter,Coalescence_Time,Eccentricity@DCO,Immediate_RLOF>CE,MT_Donor_Hist(1),MT_Donor_Hist(2),Mass(1),Mass(2),Merges_Hubble_Time,Metallicity@ZAMS(1),Optimistic_CE,Record_Type,Recycled_NS(1),Recycled_NS(2),SEED,SemiMajorAxis@DCO,Stellar_Type(1),Stellar_Type(2),Time,mixture_weight
14,1,7781.577103,0.0,0,b'2 ',b'5-8 ',0.571620,0.673707,1,0.021682,0,1,NaN,NaN,14,2.232218,11,11,405.370186,1.578707
31,1,1373.314685,0.0,0,b'2-8 ',b'5-8 ',0.773915,0.674927,1,0.004714,0,1,NaN,NaN,31,1.621585,11,11,494.155681,0.309698
40,1,64.020795,0.0,0,b'1-2 ',b'3 ',0.128751,0.345038,1,0.001365,0,1,NaN,NaN,40,0.307706,10,10,4992.556501,9.999993
52,1,233.593141,0.0,0,b'3 ',b'6 ',0.312582,0.549920,1,0.000292,0,1,NaN,NaN,52,0.692827,10,11,3276.834166,9.371884
114,1,285.668079,0.0,0,b'1-2 ',b'3 ',0.155492,0.353392,1,0.000230,0,1,NaN,NaN,114,0.480138,10,10,7039.419315,9.999954
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99883,2,1533.766025,0.0,0,b'6 ',b'3 ',1.328178,0.293608,1,0.001031,0,1,NaN,NaN,99883,1.593854,12,10,1591.331097,0.623121
99900,2,349.856371,0.0,0,b'5 ',b'3 ',0.587863,0.313331,1,0.023187,0,1,NaN,NaN,99900,0.788408,11,11,2223.960578,0.203871
99917,2,1005.769001,0.0,0,b'5 ',b'3 ',0.672699,0.305973,1,0.011928,0,1,NaN,NaN,99917,1.077492,11,10,1245.184712,0.559022
99943,1,12040.174873,0.0,0,b'2 ',b'5 ',0.593881,0.608892,1,0.001694,0,1,NaN,NaN,99943,2.429508,11,11,1121.480719,1.292016


In [43]:
# let's save this table as a csv so we can add it to the edited hdf5 file

DCO_appended_table.to_csv('Edit_DCO.csv',index=False)

In [44]:
Data_WD_edit.close()

### Let's now rewrite the hdf5 file so that we can run FastCosmicIntegration on it

In [45]:
# let's create a group that we wil called BSE_DCOs using out csv file

# Example CSV file path
csv_file = 'Edit_DCO.csv'

# Open or create an HDF5 file
with h5.File(pathToWDWD_edit_H5, 'a') as hdf_file:
    
    # deleting the previous DCO group
    del hdf_file['BSE_Double_Compact_Objects']
    print('Original BSE_Double_Compact_Objects Group deleted.')

    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv(csv_file)
    
    # Create a group (if not already created)
    group_name = 'BSE_Double_Compact_Objects'
    if group_name not in hdf_file:
        group = hdf_file.create_group(group_name)
    else:
        group = hdf_file[group_name]

   # Create datasets for each column in the CSV file
    for column in df.columns:
        dataset_name = column


        # Handle existing dataset
        if dataset_name in group:

            # Option 1: Overwrite existing dataset
            del group[dataset_name]
            dataset = group.create_dataset(dataset_name,data=df[column].to_numpy())

            # Option 2: Rename existing dataset
            # new_dataset_name = f"{dataset_name}_new"
            # dataset = group.create_dataset(new_dataset_name, data=df[column].to_numpy())

        else:
            # Create dataset and write data from DataFrame
            dataset = group.create_dataset(dataset_name, data=df[column].to_numpy())
            
            # Optionally, add attributes to the dataset (if needed)
            dataset.attrs['description'] = f'Data for column {column} from CSV file'
    
print(f'CSV data from {csv_file} added to group "{group_name}" in HDF5 file: {pathToWDWD_edit_H5}')


Original BSE_Double_Compact_Objects Group deleted.
CSV data from Edit_DCO.csv added to group "BSE_Double_Compact_Objects" in HDF5 file: /mnt/home/msantiago/ceph/data/CosmicIntegration/CosmicIntegration/COMPAS_Output_wWeights_NEW_EDIT_1.h5


In [46]:
# reading in the HDF5 file - this file is the AIS file that we should expect many WDs in
pathToWDWD_edit_H5 = '/mnt/home/msantiago/ceph/data/CosmicIntegration/CosmicIntegration/COMPAS_Output_wWeights_NEW_EDIT_1.h5' #path of the hdf5 file
Data_WD_edit = h5.File(pathToWDWD_edit_H5)

Let's check if this new group has been added to the hdf5 file

In [47]:
Data_WD_edit.keys()

<KeysViewHDF5 ['BSE_Common_Envelopes', 'BSE_Double_Compact_Objects', 'BSE_Pulsar_Evolution', 'BSE_RLOF', 'BSE_Supernovae', 'BSE_System_Parameters', 'Run_Details']>

Let's now check to see if the correct dataset names are present

In [48]:
DCOWDs = Data_WD_edit['BSE_Double_Compact_Objects'] #specifically looking at the supernovae events
list(DCOWDs.keys()) #listing the parameters recorded for each supernova

['CE_Event_Counter',
 'Coalescence_Time',
 'Eccentricity@DCO',
 'Immediate_RLOF>CE',
 'MT_Donor_Hist(1)',
 'MT_Donor_Hist(2)',
 'Mass(1)',
 'Mass(2)',
 'Merges_Hubble_Time',
 'Metallicity@ZAMS(1)',
 'Optimistic_CE',
 'Record_Type',
 'Recycled_NS(1)',
 'Recycled_NS(2)',
 'SEED',
 'SemiMajorAxis@DCO',
 'Stellar_Type(1)',
 'Stellar_Type(2)',
 'Time',
 'mixture_weight']

In [49]:
# let's open this group 
# we want to read in the bse_system_paramtetrs to get information about thes different systems
DCO_WD = pd.DataFrame() # making a pandas dataframe
for key in Data_WD_edit["BSE_Double_Compact_Objects"].keys(): #looping through the \"keys\" or paramters in BSE_System_Parameters
    DCO_WD[key] = Data_WD_edit["BSE_Double_Compact_Objects"][key][()] # adding these columns to the dataframe"

In [50]:
DCO_WD

,CE_Event_Counter,Coalescence_Time,Eccentricity@DCO,Immediate_RLOF>CE,MT_Donor_Hist(1),MT_Donor_Hist(2),Mass(1),Mass(2),Merges_Hubble_Time,Metallicity@ZAMS(1),Optimistic_CE,Record_Type,Recycled_NS(1),Recycled_NS(2),SEED,SemiMajorAxis@DCO,Stellar_Type(1),Stellar_Type(2),Time,mixture_weight
0,0,7.950392e+08,9.042606e-01,0,"b""b'2 '""","b""b'2 '""",7.621390,1.525412,0,0.000300,0,1,0.0,0.0,395,3.068217,14,13,22.738848,9.994028
1,0,1.204001e+18,1.110223e-16,0,"b""b'NA '""","b""b'NA '""",15.490306,14.294056,0,0.012003,0,1,0.0,0.0,1113,396.078766,14,14,4.874691,10.000000
2,0,2.500925e+16,3.330714e-16,0,"b""b'NA '""","b""b'NA '""",19.804426,16.017112,0,0.003526,0,1,0.0,0.0,1124,172.274778,14,14,5.295735,10.000000
3,0,1.419177e+06,1.110223e-16,0,"b""b'2 '""","b""b'2 '""",26.873803,34.481029,0,0.002050,0,1,0.0,0.0,1229,0.707168,14,14,5.500738,10.000000
4,1,1.784783e+06,0.000000e+00,0,"b""b'4 '""","b""b'5 '""",11.306701,9.799469,0,0.001517,0,1,0.0,0.0,1472,0.337254,14,14,9.005598,10.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2100,2,1.533766e+03,0.000000e+00,0,"b""b'6 '""","b""b'3 '""",1.328178,0.293608,1,0.001031,0,1,NaN,NaN,99883,1.593854,12,10,1591.331097,0.623121
2101,2,3.498564e+02,0.000000e+00,0,"b""b'5 '""","b""b'3 '""",0.587863,0.313331,1,0.023187,0,1,NaN,NaN,99900,0.788408,11,11,2223.960578,0.203871
2102,2,1.005769e+03,0.000000e+00,0,"b""b'5 '""","b""b'3 '""",0.672699,0.305973,1,0.011928,0,1,NaN,NaN,99917,1.077492,11,10,1245.184712,0.559022
2103,1,1.204017e+04,0.000000e+00,0,"b""b'2 '""","b""b'5 '""",0.593881,0.608892,1,0.001694,0,1,NaN,NaN,99943,2.429508,11,11,1121.480719,1.292016


In [51]:
DCO_WD[DCO_WD['Stellar_Type(1)']==10]

,CE_Event_Counter,Coalescence_Time,Eccentricity@DCO,Immediate_RLOF>CE,MT_Donor_Hist(1),MT_Donor_Hist(2),Mass(1),Mass(2),Merges_Hubble_Time,Metallicity@ZAMS(1),Optimistic_CE,Record_Type,Recycled_NS(1),Recycled_NS(2),SEED,SemiMajorAxis@DCO,Stellar_Type(1),Stellar_Type(2),Time,mixture_weight
54,1,64.020795,0.0,0,"b""b'1-2 '""","b""b'3 '""",0.128751,0.345038,1,0.001365,0,1,NaN,NaN,40,0.307706,10,10,4992.556501,9.999993
55,1,233.593141,0.0,0,"b""b'3 '""","b""b'6 '""",0.312582,0.549920,1,0.000292,0,1,NaN,NaN,52,0.692827,10,11,3276.834166,9.371884
56,1,285.668079,0.0,0,"b""b'1-2 '""","b""b'3 '""",0.155492,0.353392,1,0.000230,0,1,NaN,NaN,114,0.480138,10,10,7039.419315,9.999954
58,1,0.093001,0.0,0,"b""b'2 '""","b""b'3 '""",0.230328,0.257566,1,0.000392,0,1,NaN,NaN,178,0.065053,10,10,13558.528672,9.997296
60,1,1152.710025,0.0,0,"b""b'1-2 '""","b""b'3 '""",0.095871,0.347288,1,0.012881,0,1,NaN,NaN,246,0.579987,10,10,10127.255517,9.999930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1910,2,1.810362,0.0,0,"b""b'3 '""","b""b'3 '""",0.410484,0.171899,1,0.017690,0,1,NaN,NaN,89237,0.149156,10,10,4548.122007,1.562613
1947,2,421.114683,0.0,0,"b""b'3 '""","b""b'3 '""",0.407103,0.217973,1,0.010623,0,1,NaN,NaN,91146,0.627862,10,10,8320.900426,1.466733
1983,2,148.661607,0.0,0,"b""b'3 '""","b""b'3 '""",0.401303,0.214189,1,0.012640,0,1,NaN,NaN,93584,0.478274,10,10,6513.393909,1.538097
2015,2,2.611222,0.0,0,"b""b'3 '""","b""b'3 '""",0.317871,0.154917,1,0.012842,0,1,NaN,NaN,95753,0.141814,10,10,13591.803477,3.175412


The CSV would not add to the hdf5 file if I did not make the dupicate columns but let's drop them now so we can run FastCosmicIntegration on this!

In [45]:
# DCO_WD = DCO_WD.drop(['CE_Event_Counter_new','Coalescence_Time_new','Eccentricity@DCO_new','Immediate_RLOF>CE_new','MT_Donor_Hist(1)_new', \
# 'MT_Donor_Hist(2)_new','Mass(1)_new','Mass(2)_new','Merges_Hubble_Time_new','Metallicity@ZAMS(1)_new','Optimistic_CE_new','Record_Type_new',\
# 'Recycled_NS(1)_new','Recycled_NS(2)_new','SEED_new','SemiMajorAxis@DCO_new','Stellar_Type(1)_new','Stellar_Type(2)_new','Time_new','mixture_weight_new'],axis=1)

In [56]:
DCO_WD

,CE_Event_Counter,Coalescence_Time,Eccentricity@DCO,Immediate_RLOF>CE,MT_Donor_Hist(1),MT_Donor_Hist(2),Mass(1),Mass(2),Merges_Hubble_Time,Metallicity@ZAMS(1),Optimistic_CE,Record_Type,Recycled_NS(1),Recycled_NS(2),SEED,SemiMajorAxis@DCO,Stellar_Type(1),Stellar_Type(2),Time,mixture_weight
0,0,1.318672e+18,9.134209e-01,0,"b""b'NA '""","b""b'NA '""",10.459820,8.482785,0,0.000360,0,1,0.0,0.0,43,1336.614568,14,14,9.739419,10.000000
1,0,3.624469e+16,2.371437e-16,0,"b""b'NA '""","b""b'NA '""",28.885311,16.624722,0,0.001622,0,1,0.0,0.0,52,222.597147,14,14,5.297461,10.000000
2,0,2.650765e+20,1.692382e-01,0,"b""b'NA '""","b""b'NA '""",12.762462,11.971810,0,0.002880,0,1,0.0,0.0,70,1361.626335,14,14,7.190346,10.000000
3,0,2.888669e+17,6.036533e-01,0,"b""b'NA '""","b""b'NA '""",15.414209,11.071106,0,0.000270,0,1,0.0,0.0,90,374.800786,14,14,7.631813,10.000000
4,0,9.449306e+21,5.251794e-01,0,"b""b'NA '""","b""b'NA '""",15.417287,7.682713,0,0.012454,0,1,0.0,0.0,626,3967.723936,14,14,9.508827,10.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,1,9.297640e+07,7.835050e-01,0,"b""b'2 '""","b""b'4-8 '""",1.260003,1.186819,0,0.001500,0,1,1.0,0.0,373791,0.411702,13,13,55.144683,0.369471
497,1,6.076656e+03,4.515486e-01,0,"b""b'2 '""","b""b'5-8 '""",1.220009,1.222469,1,0.000488,0,1,1.0,0.0,587694,0.019758,13,13,48.745117,0.590270
498,1,4.979351e+05,7.315218e-01,0,"b""b'2 '""","b""b'5 '""",1.260000,1.223407,0,0.001254,0,1,0.0,0.0,639506,0.096072,13,13,50.429547,6.083373
499,1,2.142697e+00,2.752419e-01,0,"b""b'4-8 '""","b""b'8 '""",1.207145,1.221969,1,0.002364,0,1,1.0,0.0,992031,0.002366,13,13,40.656816,1.117100


We can now use this group instead of BSE_Double_Compact_Objects to do the rate calculation!

In [52]:
# we want to read in the bse_system_paramtetrs to get information about thes different systems
WD_SP = pd.DataFrame() # making a pandas dataframe

for key in Data_WD_edit["BSE_System_Parameters"].keys(): #looping through the "keys" or paramters in BSE_System_Parameters
    WD_SP[key] = Data_WD_edit["BSE_System_Parameters"][key][()] # adding these columns to the dataframe

In [53]:
# making sure that the m1min = 0.9
print(min(WD_SP['Mass@ZAMS(1)']))
print(max(WD_SP['Mass@ZAMS(1)']))

0.9000000460116229
141.18065235063773


In [46]:
# # we want to read in the bse_system_paramtetrs to get information about thes different systems
# DETAILS = pd.DataFrame() # making a pandas dataframe

# for key in Data_WD_edit["Run_Details"].keys(): #looping through the "keys" or paramters in BSE_System_Parameters
#     DETAILS[key] = Data_WD_edit["Run_Details"][key][()] # adding these columns to the dataframe

In [54]:
# # pd.set_option('display.max_columns', None)
# # display(DETAILS.iloc[0])
# DETAILS['metallicity-distribution']

In [61]:
Data_WD_edit.close()

In [1]:
# let's convert the columns of the data frame into numpy arrays to be able to do operations with them
time_lg = np.array(WD_SP['Time'])
t_col_lg = np.array(WD_SP['Coalescence_Time'])

t_delay_lg = [] # this will hold the delay time in Myr

for binaries_lg in range(WD_SP.shape[0]):

    delay_lg = time_lg[binaries_lg] + t_col_lg[binaries_lg] # adding the two values together to get the delay time
    t_delay_lg.append(delay_lg)

NameError: name 'np' is not defined